In [49]:
!pip install mysql-connector-python
!pip install pymysql
!pip install schedule

In [50]:
import pymysql
import pandas as pd
import os
import logging
from datetime import datetime
import schedule
import time

In [51]:
# MySQL Server Configuration - Update with your credentials
DB_CONFIG = {
    'host': 'localhost',      # e.g., 'localhost' or '127.0.0.1'
    'user': 'root',      # e.g., 'root'
    'password': 'Eliana',
    'database': 'DentalClinic'  # Change to the database you want to connect to
}

In [ ]:
csv_file = 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\daily_trans.csv'
# Configure logging
log_file = "daily_import.log"
logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(message)s")
#Function to import contents of csv file to MySQL table
def import_csv_to_mysql():
    if not os.path.exists(csv_file):
        logging.error(f"File not found: {csv_file}")
        print(f"File not found: {csv_file}")
        return
    try:
        connection = pymysql.connect(**DB_CONFIG)
        cursor = connection.cursor()
        print("Connected to MySQL!")
        # Read the CSV file
        df = pd.read_csv(csv_file)
        df.columns = df.columns.str.strip()
        # Insert data into MySQL
        for _, row in df.iterrows():
            #print(row)
            query = """
            INSERT INTO daily_transactions 
            (transaction_date, patient_id, patient_name, service_rendered, amount, payment_method, insurance_provider, status, notes)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
            """
            values = (
                row["transaction_date"],
                row["patient_id"],
                row["patient_name"],
                row["service_rendered"],
                row["amount"],
                row["payment_method"] if pd.notna(row["payment_method"]) else "Cash",
                row["insurance_provider"] if pd.notna(row["insurance_provider"]) else None,
                row["status"] if pd.notna(row["status"]) else "Paid",
                row["notes"] if pd.notna(row["notes"]) else None
            )
            cursor.execute(query, values)

        # Commit and Close Connection
        connection.commit()
        logging.info(f"✅ Successfully imported {len(df)} rows from {csv_file}")
        print(f"✅ Successfully imported {len(df)} rows from {csv_file}")
    except pymysql.MySQLError as e:
        logging.error(f"❌ MySQL Error: {e}")
        print(f"❌ MySQL Error: {e}")

    finally:
        if 'connection' in locals() and connection.open:
            cursor.close()
            connection.close()
            logging.info("🔌 MySQL connection closed.")
            print("🔌 MySQL connection closed.")

# Run the function
if __name__ == "__main__":
    import_csv_to_mysql()

# Schedule the task to run every day at midnight (00:00)
schedule.every().day.at("00:00").do(import_csv_to_mysql)

print("⏳ Scheduled job running. Press Ctrl+C to stop.")

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(60)  # Check every minute
"""
print("⏳ Scheduled job running every minute. Press Ctrl+C to stop.")

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(1)  # Check every second"""



        